In [20]:
%run "00_project_config.ipynb"

import pathlib as pl
import re as re
import collections as col
import pandas as pd
import datetime as dt

eav_store = pl.Path("../samples/vrk_samples.eav.tsv").resolve()
row_store = pl.Path("../samples/vrk_samples.tsv").resolve()

fofn_table_file = pl.Path("../samples/fofn_table.tsv")

fofn_table = pd.read_csv(fofn_table_file, sep="\t", header=0, comment="#")

special_runs = {
    "HG00733_hifi_fastq.hgsvc-UW-2021.fofn": ["HG00733red1", "HG00733red2"],
    "HG00733_ont_fastq.hgsvc-JAX-2022.fofn": ["HG00733red1", "HG00733red2"],
    "HG00733_ont_fastq.hgsvc-UW-2023.fofn": ["HG00733red2"]
}

sample_records = col.defaultdict(dict)
for (sample, read_type), fofn_files in fofn_table.groupby(["sample", "read_type"]):
    
    merged_fofns = ",".join(sorted(fofn_files["fofn_path"].values))
    sample_records[sample][read_type] = merged_fofns
    sample_records[sample]["target"] = "unphased"
    
    if sample == "HG00733":
        for fofn_file in fofn_files["fofn_path"].values:
            fofn_name = fofn_file.split("/")[-1]
            try:
                special_samples = special_runs[fofn_name]
            except KeyError:
                continue
            for special_sample in special_samples:
                try:
                    sample_records[special_sample][read_type] += "," + fofn_file
                except KeyError:
                    sample_records[special_sample][read_type] = fofn_file
                sample_records[special_sample]["target"] = "unphased"

sample_records = pd.DataFrame.from_records(sample_records)
sample_records = sample_records.transpose()
sample_records.reset_index(drop=False, inplace=True)
sample_records.rename({"index": "sample"}, inplace=True, axis=1)

sample_records.sort_values(["sample", "hifi", "ont"], inplace=True)

sample_records[["sample", "target", "hifi", "ont"]].to_csv(row_store, sep="\t", header=True, index=False)

# df = pd.DataFrame.from_records(sorted(records), columns=["sample", "key", "value"])
# df.to_csv(eav_store, header=True, index=False, sep="\t")

# def make_csv(values):
#     return ",".join(sorted(map(str, values)))

# df = df.pivot_table(
#     index="sample",
#     columns="key",
#     values="value",
#     aggfunc=make_csv
# )
# df = df[["hgsvc_sample_num", "target", "hifi", "ont", "batch"]]
# df.to_csv(row_store, header=True, index=True, sep="\t")
